There is need to create an apartment feature(i.e. building in same vicinity). I think flats in same apartment should fetch almost equal value. Also it will help us to clean data as well.

Here I am introducing new feature called apartment. It is obvious that if house is in same area , and same distance from metro station should be in same apartment building. Almost 80% of the house will fall in this category.

e.g. Nekrasovka3.776835959 got 977 houses same distance from metro. see image, there are large apartment blocks in that area.

Now this will help us to clean data as well. id 18344 got full_sq as 634. Now may houses of Nekrasovka and with the same distance from metro are there. But none of them got such huge full_sq. however there are many house in same area with 63 square meters. so this must be typo and we can clean this value. Similarly we can populate missing but important feature like floor, max floor and life_sq from there.

![Map of Nekrasovka][1]


  [1]: https://www.google.co.in/maps/place/r-n+Nekrasovka,+Moscow,+Russia/@55.7031455,37.9308046,2749m/data=!3m1!1e3!4m5!3m4!1s0x414ac9cf2ba87c27:0x1a347d594875dd2!8m2!3d55.6828266!4d37.9441291

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
macro = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])

In [ ]:
# Merging macro data with train and test
train = pd.merge(train, macro, how='left', on='timestamp')
test = pd.merge(test, macro, how='left', on='timestamp')

In [ ]:
# normalize prize feature and drop it
train["price_doc"] = np.log1p(train["price_doc"])
# store it as Y
Y_train = train["price_doc"]
# Dropping price column
train.drop("price_doc", axis=1, inplace=True)

In [ ]:
# Merging both dataframes
all_data = pd.concat((train,test))

In [ ]:
# Creating new apartment_name and month year feature
all_data['apartment_name'] = all_data["sub_area"] + all_data["metro_km_avto"].astype(str)
all_data["month_year"]=all_data.timestamp.dt.month + all_data.timestamp.dt.year * 100
# year and week #
all_data["week_year"] = all_data["timestamp"].dt.year*100 + all_data["timestamp"].dt.weekofyear


Problem here is not just about missing values, but incorrect/dirty values as well. We need to identify outliers individually from each important feature. So let's first start with most important column full_sq

In [ ]:
fig, ax = plt.subplots(figsize=(10,12))
sns.boxplot(y="full_sq",data=all_data)
ax.set_title("Box chart of full_sq")

So we can clearly see some outliers there. Let's use three sigma to identify those.

In [ ]:
print (all_data.dtypes)

In [ ]:
all_data['week_year']=all_data['week_year'].apply(str)

In [ ]:
full_sq_mean = np.mean(all_data.full_sq, axis=0)
full_sq_std = np.std(all_data.full_sq, axis=0)
full_sq_mean + 6 * full_sq_std

In [ ]:
#Scatter plot
fig, ax = plt.subplots(figsize=(10,12))
fig = ax.scatter(x="week_year",y="full_sq",data=all_data[all_data.full_sq<5000])
plt.xlabel('yearweek', fontsize=12)

#ax.set_title("Scatter to identify outliers")
'''
plt.figure(figsize=(12,8))
sns.pointplot(x='yearweek', y='price_doc', data=train_df)
plt.ylabel('price_doc', fontsize=12)
plt.xlabel('yearweek', fontsize=12)
plt.title('Median Price distribution by year and week_num')
plt.xticks(rotation='vertical')
plt.show()'''

In [ ]:
all_data[(all_data.full_sq<6) & (all_data.full_sq>300)]['apartment_name']

In [ ]:
import scipy.stats.mstats as mstats
def mode(x):
    return mstats.mode(x, axis=None)[0]

In [ ]:
g_Apartment_col.columns= ['apartment_name','col_mode'] 
g_Apartment_col=all_data.groupby('apartment_name').full_sq.value_counts()


In [ ]:
all_data.groupby('apartment_name')['full_sq'].agg(['mode'])[0]

In [ ]:
all_data.groupby(['apartment_name']).apply(pd.DataFrame.mode).reset_index(drop=True)

In [ ]:
g_Apartment_col=all_data.groupby('apartment_name')['full_sq'].agg(['mean','std','mode'])[0].reset_index()
g_Apartment_col.columns= ['apartment_name','col_mean','col_std','col_mode'] 


In [ ]:
all_data[all_data.apartment_name=='Ljublino1.686566142']

In [ ]:
#Scatter plot
fig, ax = plt.subplots(figsize=(10,12))
fig = sns.regplot(x="timestamp.dt",y="full_sq",data=all_data)
ax.set_title("Scatter to identify outliers")